# Biodiversity Intactness Index change in Phoenix, AZ
### Author: Austin Martinez
### GitHub Repository: https://github.com/austinmartinez12/urbanization-bii-mapping

## About Section

#### Purpose: 


#### Highlights: 


#### About the data:


## Loading in data

In [ ]:
# Load in Libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rio
import matplotlib.pyplot as plt
import contextily as ctx

# Load in stack libraries
import pandas as pd
import planetary_computer
import pystac_client
import rich.table
import stackstac
from geogif import gif

In [ ]:
# Access the io-biodiversity collection from the Microsoft Planetary Computer STAC catalog to collect 2017 and 2020 rasters covering the Phoenix subdivision.
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

bbox_of_interest = [-112.826843, 32.974108, -111.184387, 33.863574]
search = catalog.search(collections=["io-biodiversity"], bbox=bbox_of_interest)

items = list(search.items())
for item in items:
    print(item)

asset_table = rich.table.Table("Asset Key", "Asset Title")
for key, value in items[-1].assets.items():
    asset_table.add_row(key, value.title)
asset_table

property_table = rich.table.Table("Property Name", "Property Value")
for key, value in sorted(items[-1].properties.items()):
    property_table.add_row(key, str(value))
property_table

stack = (
    stackstac.stack(items, bounds_latlon=bbox_of_interest, assets=["data"])
    .assign_coords(
        time=pd.to_datetime([item.properties["start_datetime"] for item in items])
        .tz_convert(None)
        .to_numpy()
    )
    .sortby("time")
)
stack.name = "Biodiversity Intactness"
stack

<Item id=bii_2020_34.74464974521749_-115.38597824385106_cog>
<Item id=bii_2019_34.74464974521749_-115.38597824385106_cog>
<Item id=bii_2018_34.74464974521749_-115.38597824385106_cog>
<Item id=bii_2017_34.74464974521749_-115.38597824385106_cog>


<xarray.DataArray 'Biodiversity Intactness' (time: 4, band: 1, y: 991, x: 1830)> Size: 58MB
dask.array<getitem, shape=(4, 1, 991, 1830), dtype=float64, chunksize=(1, 1, 991, 1024), chunktype=numpy.ndarray>
Coordinates: (12/15)
    id              (time) <U50 800B 'bii_2017_34.74464974521749_-115.3859782...
  * band            (band) <U4 16B 'data'
  * x               (x) float64 15kB -112.8 -112.8 -112.8 ... -111.2 -111.2
  * y               (y) float64 8kB 33.86 33.86 33.86 ... 32.98 32.98 32.97
    proj:shape      object 8B {7992}
    proj:epsg       int64 8B 4326
    ...              ...
    description     <U54 216B 'Terrestrial biodiversity intactness at 100m re...
    raster:bands    object 8B {'sampling': 'area', 'data_type': 'float32', 's...
    title           <U23 92B 'Biodiversity Intactness'
    version         <U2 8B 'v1'
    epsg            int64 8B 4326
  * time            (time) datetime64[ns] 32B 2017-01-01 ... 2020-01-01
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(-112.82750137012779, 32.973560...
    crs:         epsg:4326
    transform:   | 0.00, 0.00,-112.83|\n| 0.00,-0.00, 33.86|\n| 0.00, 0.00, 1...
    resolution:  0.0008983152841195215

In [4]:
# Create a reproducible file path
az_path = os.path.join("data", "cb_2018_04_cousub_500k", "cb_2018_04_cousub_500k.shp")
# Read in census data for Arizona
az_subdiv = gpd.read_file(az_path)
# Select the Phoenix Subdivision
phoenix = az_subdiv[az_subdiv["NAME"] == "Phoenix"]